In [2]:
# Importing required packages
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine
import re
import time
import datetime
from datetime import datetime
import socket
import getpass

#required packages
import mysql.connector as connection
from functools import reduce

In [3]:
st = []
ed = []
du = []
date_range= []
v_host_name = []

pd.set_option('display.max_column', None)

start = datetime.now()
st.append(start)

## Final Function
def generate_patient_listing_report(p_start_date, p_end_date):
    
    try:
        v_visit_started_date = p_start_date
        v_visit_ended_date = p_end_date
        
        def f_connect_to_database(p_host, p_user, p_password, p_database):
            while True:
                try:
                    # Setup database connection
                    v_mydb = connection.connect(
                        host=p_host,
                        user=p_user,
                        password=p_password,
                        database=p_database,
                        use_pure=True
                    )
                    v_cursor = v_mydb.cursor()
                    # print("Connected to the database\n")
                    return v_mydb, v_cursor

                except connection.Error as err:
                    print(f"Error: {err}")
                    print("\n retry !")
                    return None, None


        # Function to close the database connection
        def f_close_database_connection(p_mydb, p_cursor):
            try:
                if p_cursor is not None:
                    p_cursor.close()
                    # print("Cursor closed.")
                if p_mydb.is_connected():
                    p_mydb.close()
                    # print("Database connection closed.\n")
            except Exception as e:
                return f"An error occurred while closing the database connection: {e}"

        while True:
            v_host_input = input("Enter the host: ")
            v_user_input = input("Enter the username: ")
            v_password_input = getpass.getpass("Enter the password: ")
            v_database_input = input("Enter the database: ")
            v_mydb,v_cursor = f_connect_to_database(v_host_input, v_user_input, v_password_input, v_database_input)

            if v_mydb is not None:

                #Close the database connection
                f_close_database_connection(v_mydb, v_cursor)
                break


        # SQL queries
        v_queries = {'Query1':'''SELECT p.patient_id AS patient_id,
                                        max(pi.identifier) AS openmrs_id
                                   FROM patient p
                                        JOIN patient_identifier pi ON p.patient_id = pi.patient_id AND pi.identifier_type = 3 AND pi.voided = 0  
                                        where p.date_created between %s and %s
                                        group by patient_id;''',

                     'Query2':'''SELECT p.patient_id AS patient_id,
                                        p.date_created AS patient_creation_date,
                                        u.username AS CHW_name,
                                        CONCAT_WS(' ', pn.given_name, pn.family_name) AS Beneficiary_Name,
                                        pe.gender AS Gender,
                                        FLOOR(DATEDIFF(CURRENT_DATE(), pe.birthdate) / 365.25) AS Age
                                    FROM patient p
                                        JOIN person pe ON p.patient_id = pe.person_id AND pe.voided = 0
                                        JOIN person_name pn ON pe.person_id = pn.person_id AND pn.preferred = 1
                                        JOIN users u ON p.creator = u.user_id AND u.retired = 0 and p.date_created between %s and %s;''',
                     
                    'Query3':'''select pi.patient_id,
                                        l.location_id,
                                        l.name as Village,
                                	    ls.name as Sanch,
                                		l.county_district as District,
                                        l.state_province as State,
                                		l.country as Country,
                                        l.postal_code as Postal_code
                                 from  patient_identifier pi 
                                 join location l on (pi.voided = 0 and pi.location_id = l.location_id)
                                 left join location ls on (ls.location_id = l.parent_location)
                                 where pi.date_created between %s and %s;''',
                     
                     'Query4':'''select person_id, 
                                        country as patient_country,
                                        state_province as patient_state,
                                        county_district as patient_district,
                                        city_village as patient_village,
                                        address1 as patient_address,
                                        address2 as patient_address1,
                                        postal_code as patient_pincode 
                                        from person_address where preferred = 1 and voided = 0 
                                        and date_created between %s and %s;''',
                     
                     'Query5':'''SELECT pa.person_id, pat.person_attribute_type_id, pat.name, pa.value 
                                        FROM person_attribute pa 
                                        JOIN person_attribute_type pat ON (pa.person_attribute_type_id = pat.person_attribute_type_id)
                                        join patient p on (pa.voided=0 and pa.person_id = p.patient_id and p.voided=0)
                                        where pa.voided = 0 AND pat.retired=0 AND pat.edit_privilege = 'Add Reports' 
                                        and p.date_created between %s and %s;'''}
        
        def f_get_date_range_from_tables(p_mydb, p_cursor):
                    try:
                        if p_mydb.is_connected():
                            # SQL query to fetch the minimum and maximum dates from "visit" table
                            date_query = "SELECT MIN(date_created) AS min_date, MAX(date_created) AS max_date FROM patient WHERE voided = 0;"
        
                            p_cursor.execute(date_query)
                            v_date_results = p_cursor.fetchall()
        
                            if v_date_results:
                                v_min_date, v_max_date = v_date_results[0]
                                return v_min_date, v_max_date
        
                            else:
                                print("No data available in the tables.")
                                return None, None
                        else:
                            print("Database connection is not opened for getting the Minnimum and Maximum dates.")
                            return None, None
                    except Exception as e:
                        print(f"An error occurred: {e}")

        
        # Connect to the database
        v_mydb, v_cursor = f_connect_to_database(v_host_input, v_user_input, v_password_input, v_database_input)

        # Get the date range from the SQL tables
        v_min_table_date, v_max_table_date = f_get_date_range_from_tables(v_mydb, v_cursor)

        # Close the database connection
        f_close_database_connection(v_mydb, v_cursor)

        # Print the date range to inform the user
        print(f"Date range available in the SQL table: {v_min_table_date} to {v_max_table_date}\n")
        
        def f_execute_query_with_params(p_mydb, p_cursor, p_query, p_params):
            try:
                if p_mydb.is_connected():
                    p_cursor.execute(p_query, p_params)
                    v_results = p_cursor.fetchall()
                    v_columns = [desc[0] for desc in p_cursor.description]
                    df = pd.DataFrame(v_results, columns=v_columns)
                    # print(df)
                    return df
                else:
                    print("Database connection is not opened.")
                    return None  # Return an empty DataFrame if no results
            except Exception as e:
                print(f"An error occurred: {e}")
                return None
        
        # Modify the f_fetch_data1 function
        def f_fetch_data1(p_query, p_params):
            v_dataframe = []
            try:
                v_params = p_params
        
                # Connect to the database and get the cursor
                v_mydb, v_cursor = f_connect_to_database(v_host_input, v_user_input, v_password_input, v_database_input)
        
                # Execute each query with input parameters and store the output in a list
                for query_name, query in v_queries.items():
                    if query_name =='Query5':
                        df = f_execute_query_with_params(v_mydb, v_cursor, query, v_params)
                        if df is not None:
                            # print('Printing:', df)
                            # Pivot the data to get each attribute value into separate columns
                           pivoted_data = df.pivot_table(index='person_id', columns='name', values='value', aggfunc='first').reset_index()
                            # print(f"pivoted_data: {pivoted_data}")
                            
                            # Merge the grouped data with the original DataFrame on person_id
                           merged_data = pd.merge(df['person_id'], pivoted_data, on='person_id', how='inner')
                            # print(f"merged: {merged_data}")
                            
                            # Drop the duplicates
                           merged_data = merged_data.drop_duplicates()
                           v_dataframe.append(merged_data)
                            # print(f'{query_name} Executed successfully...\n')
                    else:
                        df = f_execute_query_with_params(v_mydb, v_cursor, query, v_params)
                        if df is not None:
                            v_dataframe.append(df)
                            # print(f"{query_name} executed successfully.\n")
        
                # Close the database connection
                f_close_database_connection(v_mydb, v_cursor)
        
                # Merge the DataFrames
                if len(v_dataframe)==1:
                    return v_dataframe[0]
                elif len(v_dataframe) == 2:
                    # Merge DataFrames from the list
                    v_data = pd.merge(v_dataframe[0], v_dataframe[1], on='patient_id', how='outer')  # Merge the first two DataFrames
                    return v_data
                else:
                    # Merge the first two DataFrames
                    v_data = pd.merge(v_dataframe[0], v_dataframe[1], on='patient_id', how='outer')
                    v_data = pd.merge(v_data, v_dataframe[2], on='patient_id', how='outer') 
                    v_data = pd.merge(v_data, v_dataframe[3], left_on='patient_id', right_on='person_id', how='inner') 
                    v_data = pd.merge(v_data, v_dataframe[4], on='person_id', how='outer')
                    return v_data
        
            except Exception as e:
                print('In this function', e)

        # Funciton for filtering the data between two days
        def f_filter_data(p_start_date, p_end_date, p_query):
            v_queries = []

            v_start_date1 = p_start_date
            v_end_date1 = p_end_date

            for i in range(len(p_query)):
                v_dummy = p_query[i].format(v_start_date1, v_end_date1)
                v_queries.append(v_dummy)

            v_data = f_fetch_data1(v_queries)
            return v_data
        
        ## SQL query to fetch the minimum and maximum dates from "visit" table
        v_min_date = v_min_table_date
        v_last_date = v_max_table_date
        v_hours = v_min_date.hour
        v_minutes = v_min_date.minute
        v_seconds = v_min_date.second
    
        # print(f'Date Between **Starting Date - ({v_min_date}) and **End Date - ({v_last_date}).\n\n**The format is yyy-mm-dd\n')
    
        try:
            while True:
                # Getting input from user
                v_strt_date_input = v_visit_started_date
                try:
                    datetime_object = pd.to_datetime(v_strt_date_input, format='%Y-%m-%d')
                    v_start_date = datetime_object.strftime('%Y-%m-%d') + ' 00:00:00'
                    print("\nDate Entered: \t", datetime_object)
                    break
                except Exception as e:
                        print("An error occurred, Enter date in YYYY-MM-DD format:", e)

            try:
                while True:
                    v_end_date_input = v_visit_ended_date
                    try:  
                        datetime_object1 = pd.to_datetime(v_end_date_input, format='%Y-%m-%d')
                        v_end_date = datetime_object1.strftime('%Y-%m-%d') +' 23:59:59'
                        # v_end_date = datetime_object1.strftime('%Y-%m-%d %H:%M:%S')
                        print("\nDate Entered: \t", datetime_object1, '\n')
                        break
                    except Exception as e:
                        print("An error occurred, Enter date in YYYY-MM-DD format:", e)

                if (datetime_object1 <= v_last_date) and (datetime_object1 >= datetime_object):
                
                    # Getting the username
                    v_hostname = socket.gethostname()
                    v_host_name.append(v_hostname)
                    
                    # Execute each query with input parameters
                    v_params = (v_start_date, v_end_date)
                                                            
                    # Entered data ranges
                    date_range.append(v_start_date+' / '+v_end_date)
                    
                    # Passing the startdate, end date and query in the below function to fetch the data
                    v_sample = f_fetch_data1(v_queries, v_params)
                    v_sample = v_sample.drop_duplicates()
                    v_sample = v_sample.drop('person_id', axis=1)
                    
                    # print('\nPatient listing between 2 dates dataframe created\n')
                    
                    # Get the current date and time
                    file_name = datetime.now()
                    # Format the datetime object
                    formatted_file_name = int(file_name.strftime("%Y%m%d%H%M%S"))

                    # Conver the dataframe into an excel file
                    v_sample.to_excel(f'Patient_listing_{formatted_file_name}.xlsx', index=False)
                    
                    print(f'\nPatient_listing_b/w_2_dates_{formatted_file_name}.xlsx Excel Output Created...:)\n')
                    
                    end = datetime.now()
                    ed.append(end)

                    duration = ed[-1]-st[-1]
                    du.append(duration)

                    dur = {'Host_name':v_host_name, 'Date_Range':date_range, 'Start': st, 'End':ed, 'Duration':du}
                    duration_df = pd.DataFrame(dur)

                    # Convert the 'Duration' column to seconds
                    duration_df['Duration'] = duration_df['Duration'].dt.total_seconds()
                    duration_df.to_excel('Duration2.xlsx', index=False)
                    
                    #print('Executed successfully ')
                    
                    # Return the file dataframe and the duration dataframe
                    return v_sample, duration_df
                    
                else:
                    print('Enter the date with range')
                    
            except Exception as e:
                print('Error 2', e)
                #     break
                # break
                #     # else:
                    #     print('Error3')
                    #break
            # except Exception as e:
            #     print(e)
        except Exception as e:
            print('Error 4', e)
    except Exception as e:
        print('Error in the main function: ', e)

In [4]:
v_patient_list = generate_patient_listing_report('2024-04-28' , '2024-05-21')

Enter the host:  dev.intelehealth.org
Enter the username:  root
Enter the password:  ········
Enter the database:  openmrs


Date range available in the SQL table: 2024-04-28 11:29:12 to 2024-05-21 14:13:42


Date Entered: 	 2024-04-28 00:00:00

Date Entered: 	 2024-05-21 00:00:00 


Patient_listing_b/w_2_dates_20240521143626.xlsx Excel Output Created...:)

